In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
from collections import Counter
import seaborn as sns
import sklearn.preprocessing as preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import parallel_backend
from sklearn.ensemble import HistGradientBoostingClassifier


In [3]:
def plot_bar_dic(my_dict):
    keys = list(my_dict.keys())
    # get values in the same order as keys, and parse percentage values
    vals = [float(my_dict[k]) for k in keys]
    key_len=np.arange(len(keys))
    sns.barplot(x=key_len, y=vals)

In [4]:
train_csv=pd.read_csv('./train.csv.zip')
test_csv=pd.read_csv('./test.csv.zip')
bids_csv=pd.read_csv('./bids.csv.zip')

In [5]:
sub_temp=pd.read_csv('./sampleSubmission.csv')

In [6]:
bids_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7656334 entries, 0 to 7656333
Data columns (total 9 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   bid_id       int64 
 1   bidder_id    object
 2   auction      object
 3   merchandise  object
 4   device       object
 5   time         int64 
 6   country      object
 7   ip           object
 8   url          object
dtypes: int64(2), object(7)
memory usage: 525.7+ MB


In [7]:
def bid_per_auction(temp_bids):
    auc_counter=Counter(temp_bids['auction'])
    CC=np.array(list(auc_counter.values()))
    return CC.mean()

        
    

In [33]:
def med_bid_time(temp_bids):
    temp_time=list(temp_bids['time'])
    temp_time.sort()
    temp_time_before=np.array(temp_time[0:-1])
    temp_time_after=np.array(temp_time[1:])
    if len(temp_time_after)>0:
        return np.median(np.array(temp_time_after-temp_time_before))
    else:
        return temp_time[0]
        

In [34]:
def make_feature(train_data,bids_data):
    train_feature=pd.DataFrame()
    bid_per_auction_list=[]
    med_bid_time_list=[]
    label=[]
    for ind,name in enumerate(train_data['bidder_id']):
        mask=bids_data['bidder_id']==name
        temp_bids=bids_data[mask]
        if len(temp_bids)>0:
            bid_per_auction_list.append(bid_per_auction(temp_bids))
            med_bid_time_list.append(med_bid_time(temp_bids))
            label.append(train_data.loc[ind,'outcome'])
    train_feature['bid_per_auction'] =bid_per_auction_list
    train_feature['med_bid_time'] =med_bid_time_list
    train_feature['out']=label
            
    return train_feature
    
    

In [45]:
def make_feature_speedup(train_data,bids_data):
    train_feature=pd.DataFrame()
    bid_per_auction_list=[]
    med_bid_time_list=[]
    label=[]
    bids=bids_data.set_index(['bidder_id'])
    for ind,name in enumerate(train_data['bidder_id']):
        temp_bids=bids.loc[name]
       

    

In [46]:
temp=make_feature_speedup(train_csv,bids_csv)

KeyError: '5f50c6187a179e2ee7ba2fbcfc845c7a1smgr'

In [35]:
temp=make_feature(train_csv,bids_csv)
clf=train_classifier(temp)

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [36]:
def train_classifier(feature_data):
    x=feature_data.drop(columns=['out'])
    x= preprocessing.normalize(x.values, axis=0)
    y=feature_data['out']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    # clf=RandomForestClassifier(n_jobs=4,n_estimators=800, max_depth=None, min_samples_leaf=1, random_state=42, criterion='entropy')
    clf=HistGradientBoostingClassifier(max_iter=100).fit(X_train, y_train)
    clf.fit(X_train, y_train)
    score=clf.score(X_test, y_test)
    print(score)
    return clf
    
    

In [ ]:
def make_feature_test(test_data,bids_data):
    test_feature=pd.DataFrame()
    bid_per_auction_list=[]
    med_bid_time_list=[]
    label=[]
    for ind,name in enumerate(test_data['bidder_id']):
        mask=bids_data['bidder_id']==name
        temp_bids=bids_data[mask]
        if len(temp_bids)>0:
            bid_per_auction_list.append(bid_per_auction(temp_bids))
            med_bid_time_list.append(med_bid_time(temp_bids))
        else:
            bid_per_auction_list.append(0)
            med_bid_time_list.append(0)

    test_feature['bid_per_auction'] =bid_per_auction_list
    test_feature['med_bid_time'] =med_bid_time_list
            
    return test_feature

In [ ]:
def test_submission(test_data,bids_data,clf):
    feature=make_feature_test(test_csv,bids_csv)
    with parallel_backend('threading', n_jobs=4):
        output=clf.predict(feature)
    return output
   
    
    

In [ ]:
output=test_submission(test_csv,bids_csv,clf)

c:\Users\wtmh\anaconda3\envs\DM_term\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
CC=Counter(temp['out'])
print(CC)

Counter({0.0: 1881, 1.0: 103})


In [27]:
print(len(test_csv))

4700


In [25]:
sub=pd.DataFrame()
sub['bidder_id']=test_csv['bidder_id']
sub['prediction']=output
sub.to_csv("sub.csv",sub)

ValueError: Length of values (4630) does not match length of index (4700)

In [23]:
CC=Counter(output)
print(CC)

Counter({0.0: 4630})


In [36]:
mask=bids_csv['auction']==bids_csv.loc[10,'auction']
temp_bids_csv=bids_csv[mask]
print(len(temp_bids_csv))

17919
